# Importiamo le librerie

In [1]:
import transformers
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModel, BertTokenizer, BertModel, BertConfig

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [2]:
from torch import optim
from torch import nn
import torch.nn.functional as F
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

In [3]:
import pandas as pd
import re
import matplotlib.pyplot as plt
from tqdm import tqdm, trange
import numpy as np
import pickle

In [4]:
import seaborn as sn
from sklearn.metrics import accuracy_score, f1_score
import math
from scipy.stats import wilcoxon

**Funzione che spacchetta gli alberi**

In [5]:
def unplickle_trees(path_tree_file):
    print('--->read DTKs')
    dt_trees = []
    with open(path_tree_file, 'rb') as fr:
        try:
            while True:
                dt_trees.append(pickle.load(fr))
        except EOFError:
            pass
    return [torch.FloatTensor(i) for i in dt_trees]

# Importiamo il dataset e gli alberi

In [6]:
df1 = pd.read_csv('HR_dataset_NEW.csv')
trees1 = unplickle_trees('alberi_fine_FF.pkl')

df = df1[:30000]
trees = trees1[:30000]

del df1
del trees1

df.head()

--->read DTKs


,autore,categoria,data,patologia,rating,reparto,struttura,testo,titolo,filename,target
0,Silvia Grotto,Abrasione cerebrale,"09 Agosto, 2013",Aneurisma cerebrale tipo Blister.,"{'Voto medio': 4.3, 'Competenza': 5.0, 'Assist...",Neuroradiologia Ospedale Vicenza,Ospedale San Bortolo di Vicenza,Siamo tutti abituati a pensare ai Super Eroi d...,Neuroradiologia S.Bortolo: eccellenza,Malattie del cervello e del sistema nervoso,1
1,gilda fabiani,Abrasione cerebrale,"24 Dicembre, 2011",Abrasione cerebrale.,"{'Voto medio': 4.8, 'Competenza': 5.0, 'Assist...",Neurochirurgia Ospedale Terni,Ospedale Santa Maria di Terni,Un ringraziamento speciale al dott. Carletti e...,Operazione al cervello- dott. Carletti,Malattie del cervello e del sistema nervoso,1
2,Rescigno Andrea,Adenoma ipofisario,"15 Agosto, 2021",Macroadenoma ipofisario.,"{'Voto medio': 5.0, 'Competenza': 5.0, 'Assist...",Neurochirurgia Ospedale Caserta,Ospedale di Caserta,Desidero esprimere il mio ringraziamento a tut...,Ringraziamenti al Reparto di Neurochirurgia,Malattie del cervello e del sistema nervoso,1
3,Stan Marian,Adenoma ipofisario,"29 Luglio, 2021",Adenoma ipofisario.,"{'Voto medio': 5.0, 'Competenza': 5.0, 'Assist...",Neurochirurgia Ospedale Treviso,Ospedale S. Maria di Ca' Foncello di Treviso,Volevo ringraziare per la gentilezza e per i c...,Grazie Dott. Jacopo Del Verme,Malattie del cervello e del sistema nervoso,1
4,Lucia,Adenoma ipofisario,"23 Luglio, 2021",Macroadenoma ipofisario.,"{'Voto medio': 4.0, 'Competenza': 1.0, 'Assist...",Neurochirurgia Policlinico Gemelli,Ospedale Policlinico Agostino Gemelli di Roma,Se potessi dare 10 come voto al professore Vis...,Grazie professore M. Visocchi,Malattie del cervello e del sistema nervoso,1


In [7]:
len(df),len(trees)

(30000, 30000)

In [8]:
sentences = df.testo.values
labels = df.target.values

unique, counts = np.unique(labels, return_counts = True)

print(unique, counts)

[-1  1] [ 1843 28157]


**Funzioni utili per il calcolo statistico**

In [9]:
def calculate_mean(number_list):
    sum = 0
    for number in number_list:
        sum += number

    return sum/len(number_list)

def calculate_standard_deviation(number_list):
    mean = calculate_mean(number_list)
    summatory = 0
    for number in number_list:
        summatory += pow((number - mean),2)

    summatory = summatory/len(number_list)

    return math.sqrt(summatory)

In [10]:
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

## **Italians Bert**
* **Italian Bert**
    * Tokenizzatore -->  BertTokenizer.from_pretrained("dbmdz/bert-base-italian-uncased", do_lower_case=True)
    * Modello --> BertModel.from_pretrained('dbmdz/bert-base-italian-uncased')



* **Umberto** 
    * Tokenizzatore --> AutoTokenizer.from_pretrained("Musixmatch/umberto-commoncrawl-cased-v1")
    * Modello --> AutoModel.from_pretrained("Musixmatch/umberto-commoncrawl-cased-v1")



* **Alberto** 
    * Tokenizzatore --> AutoTokenizer.from_pretrained("m-polignano-uniba/bert_uncased_L-12_H-768_A-12_italian_alb3rt0")
    * Modello --> AutoModel.from_pretrained("m-polignano-uniba/bert_uncased_L-12_H-768_A-12_italian_alb3rt0")
    
    

* **Multilingua-Bert** 
    * Tokenizzatore --> BertTokenizer.from_pretrained('bert-base-multilingual-uncased')
    * Modello --> BertModel.from_pretrained("bert-base-multilingual-uncased")

In [11]:
def define_input(seed, random_state, sentences, model_type, epochs):
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    if torch.cuda.is_available(): torch.cuda.manual_seed_all(seed)
        
    global tokenizer
    global model_architecture
        
    sentences = ["[CLS] " + sentence + " [SEP]" for sentence in sentences]
    
    if model_type == 'Italian_Bert':
        tokenizer = BertTokenizer.from_pretrained("dbmdz/bert-base-italian-uncased", do_lower_case=True)
        model_architecture = BertModel.from_pretrained('dbmdz/bert-base-italian-uncased').to("cuda" if torch.cuda.is_available() else "cpu")

    if model_type == 'Umberto':
        tokenizer = AutoTokenizer.from_pretrained("Musixmatch/umberto-commoncrawl-cased-v1")
        model_architecture = AutoModel.from_pretrained("Musixmatch/umberto-commoncrawl-cased-v1").to("cuda" if torch.cuda.is_available() else "cpu")
                
    if model_type == 'Alberto':
        tokenizer = AutoTokenizer.from_pretrained("m-polignano-uniba/bert_uncased_L-12_H-768_A-12_italian_alb3rt0")
        model_architecture = AutoModel.from_pretrained("m-polignano-uniba/bert_uncased_L-12_H-768_A-12_italian_alb3rt0").to("cuda" if torch.cuda.is_available() else "cpu")

    if model_type == 'Multilingua-Bert':
        tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-uncased')
        model_architecture = BertModel.from_pretrained("bert-base-multilingual-uncased").to("cuda" if torch.cuda.is_available() else "cpu")

    tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
    
    MAX_LEN = 64
    input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],
                          maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
    
    attention_masks = []

    for seq in input_ids:
        seq_mask = [float(i>0) for i in seq]
        attention_masks.append(seq_mask)
        
    X_inputs, test_inputs, X_labels, test_labels = train_test_split(input_ids, labels, random_state=random_state, test_size=0.2)
    X_masks, test_masks, _, _ = train_test_split(attention_masks, input_ids, random_state=random_state, test_size=0.2)
    X_trees, test_trees, _, _ = train_test_split(trees, input_ids, random_state=random_state, test_size=0.2)

    train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(X_inputs, X_labels, random_state=random_state, test_size=0.3)
    train_masks, validation_masks, _, _ = train_test_split(X_masks, X_inputs, random_state=random_state, test_size=0.3)
    train_trees, validation_trees, _, _ = train_test_split(X_trees, X_inputs, random_state=random_state, test_size=0.3)

    train_inputs = torch.tensor(train_inputs)
    train_labels = torch.tensor(train_labels)
    train_masks = torch.tensor(train_masks)
    train_trees = torch.stack(train_trees)

    validation_inputs = torch.tensor(validation_inputs)
    validation_labels = torch.tensor(validation_labels)
    validation_masks = torch.tensor(validation_masks)
    validation_trees = torch.stack(validation_trees)

    test_inputs = torch.tensor(test_inputs)
    test_labels = torch.tensor(test_labels)
    test_masks = torch.tensor(test_masks)
    test_trees = torch.stack(test_trees)

    batch_size = 32

    train_data = TensorDataset(train_inputs, train_masks, train_trees, train_labels)
    train_sampler = RandomSampler(train_data)
    train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

    validation_data = TensorDataset(validation_inputs, validation_masks, validation_trees, validation_labels)
    validation_sampler = SequentialSampler(validation_data)
    validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

    test_data = TensorDataset(test_inputs, test_masks, test_trees, test_labels)
    test_sampler = SequentialSampler(test_data)
    test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)
    
    return train_dataloader, validation_dataloader, test_dataloader, model_architecture, device, test_labels
    

# Definiamo i modelli che utilizzeremo

### Solo Bert

In [12]:
class BertForSequenceClassification(nn.Module):
  
    def __init__(self, input_dim_bert, output_dim, model_architecture):
        super().__init__()
        self.bert = model_architecture
        
        self.dropout = nn.Dropout(0.1)
        self.sem_linear = nn.Linear(input_dim_bert, output_dim)
        
    def forward(self, x_sem, attention_mask):
        with torch.no_grad():
            pooled_output = self.bert(x_sem, attention_mask)[0][:, 0, :]  
            pooled_output = self.dropout(pooled_output)
        logits = self.sem_linear(pooled_output)

        return logits
        
    def get_activation(self, x_sem, x_synth):
        with torch.no_grad():
            x_sem = self.bert(x_sem)[0][:, 0, :]
            x_tot = torch.cat((x_sem, x_synth), 1)
            x_tot = self.synth_sem_linear(x_tot)
        out = F.log_softmax(x_tot, dim=1)
        return out

# Creiamo le funzioni di applicazione dei modelli

### Esecuzione di solo Bert

In [13]:
def execute_Bert(epochs, model_architecture, train_dataloader, validation_dataloader, test_dataloader, device, test_labels):
    Alone_model = BertForSequenceClassification(768,2, model_architecture)

    criterion = nn.CrossEntropyLoss()
    parameters = filter(lambda p: p.requires_grad, Alone_model.parameters())
    optimizer = optim.AdamW(Alone_model.parameters(), lr=2e-5)

    Alone_model.cuda()

    # Store our loss and accuracy for plotting
    train_loss_set = []
    # Number of training epochs 
    epoch = 0

    # BERT training loop
    for _ in trange(epochs, desc="Epoch"):  
        Alone_model.train()  
          # Tracking variables
        tr_loss = 0
        nb_tr_examples, nb_tr_steps = 0, 0
      # Train the data for one epoch
        for step, batch in enumerate(train_dataloader):
            #print(step, batch)
            # Add batch to GPU
            batch = tuple(t.cuda() for t in batch)
            # Unpack the inputs from our dataloader
            b_input_ids, b_input_mask,b_input_tree, b_labels = batch
            # Clear out the gradients (by default they accumulate)
            optimizer.zero_grad()
            # Forward pass

            #NB anche BertForSequenceClassification prende in input anche b_input_tree ma non li usa (solo per comodità)
            target_hat = Alone_model(b_input_ids, b_input_mask)

            loss = criterion(target_hat, b_labels)
            train_loss_set.append(loss.item())

            # Backward pass
            loss.backward()
            # Update parameters and take a step using the computed gradient
            optimizer.step()
            # Update tracking variables
            tr_loss += loss.item()
            nb_tr_examples += b_input_ids.size(0)
            nb_tr_steps += 1

        ## VALIDATION

      # Put model in evaluation mode
        Alone_model.eval()
        # Tracking variables 
        eval_loss, eval_accuracy = 0, 0
        nb_eval_steps, nb_eval_examples = 0, 0
        # Evaluate data for one epoch
        for batch in validation_dataloader:
            # Add batch to GPU
            batch = tuple(t.cuda() for t in batch)
            # Unpack the inputs from our dataloader
            b_input_ids, b_input_mask, b_input_tree, b_labels = batch
            # Telling the model not to compute or store gradients, saving memory and speeding up validation
            with torch.no_grad():

              # Forward pass, calculate logit predictions

            #NB anche BertForSequenceClassification prende in input anche b_input_tree ma non li usa (solo per comodità)
              logits = Alone_model(b_input_ids, b_input_mask)

            # Move logits and labels to CPU
            logits = logits.detach().cpu().numpy()
            label_ids = b_labels.to('cpu').numpy()
            tmp_eval_accuracy = flat_accuracy(logits, label_ids)    
            eval_accuracy += tmp_eval_accuracy
            nb_eval_steps += 1

        epoch +=1

    predictions = []
    Alone_model.eval()

    for batch in test_dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_input_tree, b_labels = batch

        with torch.no_grad():
            logits = Alone_model(b_input_ids, b_input_mask)
        logits = logits.detach().cpu().numpy()

        predictions.append(logits)

        flat_predictions = [item for sublist in predictions for item in sublist]
        flat_predictions = np.argmax(flat_predictions, axis=1).flatten()
        
    A = accuracy_score(test_labels.numpy(), flat_predictions)
    B = f1_score(test_labels.numpy(), flat_predictions, average='macro')
    C = f1_score(test_labels.numpy(), flat_predictions, average='weighted')
    D = f1_score(test_labels.numpy(), flat_predictions, average=None)
        
    return A,B,C,D

# Eseguiamo le funzioni

**Definiamo le varibili che univoche per tutti i modelli**

In [14]:
seed = [46, 23, 17, 54, 31, 54, 1, 28, 52, 27]
random_state = [1024, 3333, 1995, 2780, 3833, 1394, 779, 4787, 845, 5480]
model_architecture_list = ['Italian_Bert','Umberto','Alberto', 'Multilingua-Bert']
epochs = 2

model_architecture = model_architecture_list[2]

**Eseguiamo solo Bert**

In [15]:
Bert_accuracy_list = []
Bert_macro_list = []
Bert_weighted_list = []
Bert_other_0 = []
Bert_other_1 = []


for i in range(0, 10):
    train_dataloder, validation_dataloader, test_dataloder, model_architecture, device, test_labels = define_input(seed[i], random_state[i], sentences, model_architecture, epochs)
    
    A,B,C,D = execute_Bert(epochs, model_architecture, train_dataloder, validation_dataloader, test_dataloder, device, test_labels)
    
    Bert_accuracy_list.append(A)
    Bert_macro_list.append(B)
    Bert_weighted_list.append(C)
    Bert_other_0.append(D[0])
    Bert_other_1.append(D[1])

Epoch:   0%|          | 0/2 [00:00<?, ?it/s]


RuntimeError: CUDA error: device-side assert triggered

# Calcoliamo la media e la variazione standard dei 3 modelli

**Kermit Potter**

In [21]:
print('*********** F1 - SCORE CLASS 0')
print(round(calculate_mean(Potter_other_0), 2))
print(round(calculate_standard_deviation(Potter_other_0), 2))

print('*********** F1 - SCORE CLASS 1')
print(round(calculate_mean(Potter_other_1), 2))
print(round(calculate_standard_deviation(Potter_other_1), 2))

print('*********** ACCURACY')
print(round(calculate_mean(Potter_accuracy_list),2))
print(round(calculate_standard_deviation(Potter_accuracy_list),2))

print('*********** MACRO')
print(round(calculate_mean(Potter_macro_list),2))
print(round(calculate_standard_deviation(Potter_macro_list),2))

print('*********** WEIGHTED')
print(round(calculate_mean(Potter_weighted_list), 2))
print(round(calculate_standard_deviation(Potter_weighted_list), 2))

*********** F1 - SCORE CLASS 0
0.22
0.09
*********** F1 - SCORE CLASS 1
0.89
0.08
*********** ACCURACY
0.81
0.12
*********** MACRO
0.55
0.07
*********** WEIGHTED
0.84
0.08


**Kermit classico**

In [22]:
print('*********** F1 - SCORE CLASS 0')
print(round(calculate_mean(Kermit_other_0), 2))
print(round(calculate_standard_deviation(Kermit_other_0), 2))

print('*********** F1 - SCORE CLASS 1')
print(round(calculate_mean(Kermit_other_1), 2))
print(round(calculate_standard_deviation(Kermit_other_1), 2))

print('*********** ACCURACY')
print(round(calculate_mean(Kermit_accuracy_list),2))
print(round(calculate_standard_deviation(Kermit_accuracy_list),2))

print('*********** MACRO')
print(round(calculate_mean(Kermit_macro_list),2))
print(round(calculate_standard_deviation(Kermit_macro_list),2))

print('*********** WEIGHTED')
print(round(calculate_mean(Kermit_weighted_list), 2))
print(round(calculate_standard_deviation(Kermit_weighted_list), 2))

*********** F1 - SCORE CLASS 0
0.23
0.07
*********** F1 - SCORE CLASS 1
0.9
0.05
*********** ACCURACY
0.82
0.08
*********** MACRO
0.56
0.05
*********** WEIGHTED
0.85
0.05


**Solo Bert**

In [23]:
print('*********** F1 - SCORE CLASS 0')
print(round(calculate_mean(Bert_other_0), 2))
print(round(calculate_standard_deviation(Bert_other_0), 2))

print('*********** F1 - SCORE CLASS 1')
print(round(calculate_mean(Bert_other_1), 2))
print(round(calculate_standard_deviation(Bert_other_1), 2))

print('*********** ACCURACY')
print(round(calculate_mean(Bert_accuracy_list),2))
print(round(calculate_standard_deviation(Bert_accuracy_list),2))

print('*********** MACRO')
print(round(calculate_mean(Bert_macro_list),2))
print(round(calculate_standard_deviation(Bert_macro_list),2))

print('*********** WEIGHTED')
print(round(calculate_mean(Bert_weighted_list), 2))
print(round(calculate_standard_deviation(Bert_weighted_list), 2))

*********** F1 - SCORE CLASS 0
0.05
0.08
*********** F1 - SCORE CLASS 1
0.91
0.1
*********** ACCURACY
0.85
0.14
*********** MACRO
0.48
0.04
*********** WEIGHTED
0.84
0.09


# Verifichiamo se i risultati ottenuti sono statisticamente significativi

In [24]:
def p_test(p):
    alpha = 0.05
    if p > alpha:
        print('Same distribution (fail to reject H0)')
    else:
        print('Different distribution (reject H0)')

**Potter vs Kermit**

In [25]:
a_w, a_p = wilcoxon(Potter_accuracy_list, Kermit_accuracy_list)
m_w, m_p = wilcoxon(Potter_macro_list, Kermit_macro_list)
w_w, w_p = wilcoxon(Potter_weighted_list, Kermit_weighted_list)

In [26]:
p_test(a_p)
p_test(m_p)
p_test(w_p)

Same distribution (fail to reject H0)
Same distribution (fail to reject H0)
Same distribution (fail to reject H0)


**Potter vs Bert**

In [27]:
a_w, a_p = wilcoxon(Potter_accuracy_list, Bert_accuracy_list)
m_w, m_p = wilcoxon(Potter_macro_list, Bert_macro_list)
w_w, w_p = wilcoxon(Potter_weighted_list, Bert_weighted_list)

In [28]:
p_test(a_p)
p_test(m_p)
p_test(w_p)

Same distribution (fail to reject H0)
Different distribution (reject H0)
Same distribution (fail to reject H0)


**Kermit vs Bert**

In [29]:
a_w, a_p = wilcoxon(Kermit_accuracy_list, Bert_accuracy_list)
m_w, m_p = wilcoxon(Kermit_macro_list, Bert_macro_list)
w_w, w_p = wilcoxon(Kermit_weighted_list, Bert_weighted_list)

In [30]:
p_test(a_p)
p_test(m_p)
p_test(w_p)

Same distribution (fail to reject H0)
Different distribution (reject H0)
Same distribution (fail to reject H0)
